In [4]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import json

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import torch
from torch.utils.data import Dataset

from transformers import BertModel
from transformers import BertTokenizer
from transformers import BertPreTrainedModel
from transformers.models.bert.modeling_bert import BertEmbeddings, BertEncoder, BertPooler
from transformers.modeling_outputs import BaseModelOutputWithPoolingAndCrossAttentions

from typing import *

df_train = pd.read_json("RTE\\train.json", lines=True)
df_train.head()

,premise,hypothesis,label,idx
0,No Weapons of Mass Destruction Found in Iraq Yet.,Weapons of Mass Destruction Found in Iraq.,not_entailment,0
1,"A place of sorrow, after Pope John Paul II die...",Pope Benedict XVI is the new leader of the Rom...,entailment,1
2,Herceptin was already approved to treat the si...,Herceptin can be used to treat breast cancer.,entailment,2
3,"Judie Vivian, chief executive at ProMedica, a ...",The previous name of Ho Chi Minh City was Saigon.,entailment,3
4,A man is due in court later charged with the m...,Paul Stewart Hutchinson is accused of having s...,not_entailment,4


In [10]:
df = df_train.drop(columns=['idx'])
df.columns = ['context1', 'context2', 'label']
df.to_csv('RTE_train.csv')

In [7]:
class RTE_Dataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
        self.label_map = {'entailment' : 0, 'not_entailment' : 1}

    def __getitem__(self, index):
        df = self.df
        EC = self.tokenizer.encode_plus(df['context1'][index], df['context2'][index])

        input_ids = torch.tensor(EC['input_ids'])
        mask = torch.tensor(EC['attention_mask'])
        token = torch.tensor(EC['token_type_ids'])
        label = self.label_map[df['label'][index]]

        return input_ids, mask, token, label
    
    def __len__(self):
        return len(self.df)

In [8]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_batch(sample): #sample is List
    input_ids_batch = [s[0] for s in sample]
    mask_batch = [s[1] for s in sample]
    token_batch = [s[2] for s in sample]
    Label_batch = torch.tensor([s[3] for s in sample])


    input_ids_batch = pad_sequence(input_ids_batch, batch_first=True)
    mask_batch = pad_sequence(mask_batch, batch_first=True)
    token_batch = pad_sequence(token_batch, batch_first=True)

    return input_ids_batch, mask_batch, token_batch, Label_batch

BATCH_SIZE = 2

In [9]:
train_dataset = RTE_Dataset(df)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

data = next(iter(train_loader))
data

(tensor([[  101,  1109,  1418, 27629, 27109,  1116,  6313,  5460,  2499,  2142,
           1112,   170,  1684,   118,  1705, 14081,  1150,  1423,   118,  3541,
           1193,  1814,  1205, 25788,  2118,  1116,   153, 12674,   117,   170,
          24482,   118,  1214,   118,  1385,  1498,  6800,  3085,  1115,  2375,
           7845,  1103, 20395,  8755,   119,   102, 25788,  2118,  1116,  1108,
           2855,   112,   188,  3778,  6800,  3085,   119,   102,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0],
         [  101,  1135,  1144,  1151,  2212,  1765,  1201,  1290,  